In [49]:
! cp /Users/alex/Documents/bagel-data/allowed_mutants.txt . 

In [67]:
# enter your params

read_dir = './1168726/'
wt_pep = '../../../../oligo_design/bglb_model.fa'
allowed_mutants = 'allowed_mutants.txt' 
sample_list = []

with open( allowed_mutants ) as fn:
    allowed_mutants = fn.read().split()

In [68]:
# convert all the reads to FASTQ

from Bio import SeqIO
from glob import glob 

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

In [69]:
import itertools
from glob import glob 
from subprocess import call, check_output

z = zip( 'ABCD', 'EFGH' )

for ( fwd_row, rev_row, ), col in itertools.product( z, [ str(i).zfill(2) for i in range( 1, 13 ) ] ):
    
    fwd_row = fwd_row + col 
    rev_row = rev_row + col 
    
    fwd = glob( '{}/*T7_*{}.fastq'.format( read_dir, fwd_row ) ) 
    rev = glob( '{}/*T7term_*{}.fastq'.format( read_dir, rev_row ) )
    
    # N220W_T7term_Plate_Plate 01_E07.fastq
    
    print( fwd, rev ) 
        
    if len( fwd ) == len( rev ) == 1:
        sample = fwd[0].split( '/' )[2].split( '_' )[0]
        filename = fwd[0].split( '/' )[2]
        pear_opts = [ '-e', '2' ] 
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', '{}/{}'.format( read_dir, filename ) ] + pear_opts
        call( cmd )

['./1168726/H119S_T7_Plate_Plate 01_A01.fastq'] ['./1168726/H119S_T7term_Plate_Plate 01_E01.fastq']
['./1168726/H119S_T7_Plate_Plate 01_A02.fastq'] ['./1168726/H119S_T7term_Plate_Plate 01_E02.fastq']
['./1168726/H119S_T7_Plate_Plate 01_A03.fastq'] ['./1168726/H119S_T7term_Plate_Plate 01_E03.fastq']
['./1168726/R359E_T7_Plate_Plate 01_A04.fastq'] ['./1168726/R359E_T7term_Plate_Plate 01_E04.fastq']
['./1168726/R359E_T7_Plate_Plate 01_A05.fastq'] ['./1168726/R359E_T7term_Plate_Plate 01_E05.fastq']
['./1168726/R359E_T7_Plate_Plate 01_A06.fastq'] ['./1168726/R359E_T7term_Plate_Plate 01_E06.fastq']
['./1168726/N220W_T7_Plate_Plate 01_A07.fastq'] ['./1168726/N220W_T7term_Plate_Plate 01_E07.fastq']
['./1168726/N220W_T7_Plate_Plate 01_A08.fastq'] ['./1168726/N220W_T7term_Plate_Plate 01_E08.fastq']
['./1168726/N220W_T7_Plate_Plate 01_A09.fastq'] ['./1168726/N220W_T7term_Plate_Plate 01_E09.fastq']
['./1168726/N404Q_T7_Plate_Plate 01_A10.fastq'] ['./1168726/N404Q_T7term_Plate_Plate 01_E10.fastq']


In [70]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [74]:
allowed_mutants

['A109A',
 'A109C',
 'A109D',
 'A109E',
 'A109F',
 'A109G',
 'A109H',
 'A109I',
 'A109K',
 'A109L',
 'A109M',
 'A109N',
 'A109P',
 'A109Q',
 'A109R',
 'A109S',
 'A109T',
 'A109V',
 'A109W',
 'A109Y',
 'A145A',
 'A145C',
 'A145D',
 'A145E',
 'A145F',
 'A145G',
 'A145H',
 'A145I',
 'A145K',
 'A145L',
 'A145M',
 'A145N',
 'A145P',
 'A145Q',
 'A145R',
 'A145S',
 'A145T',
 'A145V',
 'A145W',
 'A145Y',
 'A168A',
 'A168C',
 'A168D',
 'A168E',
 'A168F',
 'A168G',
 'A168H',
 'A168I',
 'A168K',
 'A168L',
 'A168M',
 'A168N',
 'A168P',
 'A168Q',
 'A168R',
 'A168S',
 'A168T',
 'A168V',
 'A168W',
 'A168Y',
 'A179A',
 'A179C',
 'A179D',
 'A179E',
 'A179F',
 'A179G',
 'A179H',
 'A179I',
 'A179K',
 'A179L',
 'A179M',
 'A179N',
 'A179P',
 'A179Q',
 'A179R',
 'A179S',
 'A179T',
 'A179V',
 'A179W',
 'A179Y',
 'A188A',
 'A188C',
 'A188D',
 'A188E',
 'A188F',
 'A188G',
 'A188H',
 'A188I',
 'A188K',
 'A188L',
 'A188M',
 'A188N',
 'A188P',
 'A188Q',
 'A188R',
 'A188S',
 'A188T',
 'A188V',
 'A188W',
 'A188Y',


In [72]:
'E177D' in allowed_mutants 

True

In [77]:
# diff the assemblies against the wild type protein sequence 
singles = []
for assembly in glob( '{}/*.assembled.fasta'.format( read_dir ) ):
    name = assembly.split( '/' )[2].split( '.' )[0] 
    lines = check_output( [ 'blastx', '-subject', wt_pep, '-query', assembly, '-outfmt', '6 sseq qseq length' ] ).split()
    if lines and lines[0] and lines[1] and lines[2]:
        z = zip( str(lines[0]), str(lines[1]) )
        d = [ '{}{}{}'.format( i, n-1, j ) for n, ( i, j ) in enumerate( z ) if i != j ]
        dstr = '+'.join( d ) 
        if len( d ) == 0: 
            print( name, '\twild type' )
        elif len( d ) == 1:
            if d[0] in allowed_mutants:
                print( name, '\t{}'.format( dstr )  )
                singles.append( ( d[0], name ) )
            else:
                #print( name, '\tbad alignment' )
                print( name, '\t{} (bad alignment?)'.format( dstr )  )
        elif len( d ) > 1:
            #print name, '\t... ({} mutations)'.format( len( d ) ) 
            print( name, '\t{}'.format( dstr ) )

E177D 	E177D
E177D_T7_Plate_Plate 01_B06 	E177D
E177D_T7_Plate_Plate 01_B07 	R155S+E177D
E177D_T7_Plate_Plate 01_B08 	E177D
E177N 	G153C+E177N
E177N_T7_Plate_Plate 01_C12 	wild type
E177N_T7_Plate_Plate 01_D01 	E177N
E177N_T7_Plate_Plate 01_D02 	G153C+E177N
E406N 	wild type
E406N_T7_Plate_Plate 01_C09 	wild type
H119D 	H119D
H119D_T7_Plate_Plate 01_C01 	H119D+N443X
H119D_T7_Plate_Plate 01_C02 	H119D
H119S 	H119T+Q134P
H119S_T7_Plate_Plate 01_A01 	H119S
H119S_T7_Plate_Plate 01_A03 	H119T+Q134P
H178F 	H178F
H178F_T7_Plate_Plate 01_B01 	H178F
H178F_T7_Plate_Plate 01_B03 	H178F
H178K 	H178K
H178K_T7_Plate_Plate 01_C10 	H178K
H178K_T7_Plate_Plate 01_C11 	H178K
H315D 	Q134P+H315D
H315D_T7_Plate_Plate 01_B04 	Q134P+H315D
N220W 	T143N+N220W
N220W_T7_Plate_Plate 01_A07 	F152V+N220W
N220W_T7_Plate_Plate 01_A09 	T143N+N220W
N404E 	N404E
N404E_T7_Plate_Plate 01_D03 	N404E
N404E_T7_Plate_Plate 01_D05 	N404E
N404Q 	N404Q
N404Q_T7_Plate_Plate 01_A10 	wild type
N404Q_T7_Plate_Plate 01_A11 	N404Q
N404Q

In [78]:
dict( singles ) 

{'E177D': 'E177D_T7_Plate_Plate 01_B08',
 'E177N': 'E177N_T7_Plate_Plate 01_D01',
 'H119D': 'H119D_T7_Plate_Plate 01_C02',
 'H119S': 'H119S_T7_Plate_Plate 01_A01',
 'H178F': 'H178F_T7_Plate_Plate 01_B03',
 'H178K': 'H178K_T7_Plate_Plate 01_C11',
 'N404E': 'N404E_T7_Plate_Plate 01_D05',
 'N404Q': 'N404Q_T7_Plate_Plate 01_A12',
 'S16Q': 'S16Q_T7_Plate_Plate 01_D06',
 'W325Y': 'W325Y_T7_Plate_Plate 01_B11'}

In [81]:
# results 

picks = dict( singles ) 
picks

{'E177D': 'E177D_T7_Plate_Plate 01_B08',
 'E177N': 'E177N_T7_Plate_Plate 01_D01',
 'H119D': 'H119D_T7_Plate_Plate 01_C02',
 'H119S': 'H119S_T7_Plate_Plate 01_A01',
 'H178F': 'H178F_T7_Plate_Plate 01_B03',
 'H178K': 'H178K_T7_Plate_Plate 01_C11',
 'N404E': 'N404E_T7_Plate_Plate 01_D05',
 'N404Q': 'N404Q_T7_Plate_Plate 01_A12',
 'S16Q': 'S16Q_T7_Plate_Plate 01_D06',
 'W325Y': 'W325Y_T7_Plate_Plate 01_B11'}